## Convert behavior data

In [12]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('paper')
sns.set_style('ticks')

In [13]:
import numpy as np
import pandas as pd
from glob import glob
from os.path import join, exists
from shutil import copytree
import json

###Set directory and session information

In [14]:
directory = '/tier2/freeman/Nick/lfov.calibration'

In [15]:
key = '2016-04-24'
name = 'anm-0330549-6'

Check if behavior data exists

In [16]:
path = join(directory, 'unprocessed', 'behavior', key, name)
print exists(path)

True


###Load trial data

In [17]:
files = glob(join(path, 'trial-*.csv'))
files.sort()
trials = [pd.read_csv(x) for x in files]

Create session table

In [18]:
session = pd.concat(trials)
session.reset_index(inplace=True)
session.rename(columns={'index': 'tick'}, inplace=True)

Add global time

In [19]:
trial = session.groupby(['number']).last()
offsets = np.concatenate(([0],trial.time.values))
offsets = np.cumsum(offsets)
session['allTime'] = session.time + offsets[session.number]

Add filtered speed

In [20]:
speed = (session.xSpeed.pow(2)+session.ySpeed.pow(2)).pow(.5)
session['speed'] = np.convolve(speed,np.ones(50)/50, mode='same')    

### Save full-length session

In [21]:
session.to_csv(join(path, 'session.csv'))

###Copy data to new directory

In [22]:
savepath = join(directory, 'reprocessed', name, key, 'behavior')
copytree(path,savepath)